In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import platform
import os
import glob

# ==========================================
# 0. 설정 및 폰트
# ==========================================
ORIGINAL_DATA_PATH = "data/final_features.csv"
PREDICTION_DIR = "output/train_model"
OUTPUT_DIR = "output/visualize_performance"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def set_korean_font():
    system_name = platform.system()
    if system_name == 'Darwin': plt.rcParams['font.family'] = 'AppleGothic'
    elif system_name == 'Windows': plt.rcParams['font.family'] = 'Malgun Gothic'
    else: plt.rcParams['font.family'] = 'NanumGothic'
    plt.rcParams['axes.unicode_minus'] = False

# ==========================================
# 1. 데이터 로드 및 병합
# ==========================================
def load_and_merge_data(position):
    try:
        search_pattern = os.path.join(PREDICTION_DIR, f"test_predictions_{position}_*.csv")
        files = glob.glob(search_pattern)
        if not files:
            print(f"⚠️ {position}: 예측 파일을 찾을 수 없습니다.")
            return pd.DataFrame()
        latest_file = max(files, key=os.path.getctime)

        print(f"📂 [{position}] Loading: {os.path.basename(latest_file)}")
        df_pred = pd.read_csv(latest_file)

        df_origin = pd.read_csv(ORIGINAL_DATA_PATH)
        df_origin = df_origin[df_origin['position'] == position]
        df_origin = df_origin[df_origin['minute'] > 0].reset_index(drop=True)

        df_pred_renamed = df_pred.rename(columns={'actualGold': 'totalGold'})
        merge_keys = ['matchId', 'totalGold', 'xp', 'minionsKilled']
        cols_to_bring = ['matchId', 'minute', 'totalGold', 'xp', 'minionsKilled', 'participantId', 'win']

        df_merged = pd.merge(
            df_pred_renamed,
            df_origin[cols_to_bring],
            on=merge_keys,
            how='inner'
        )

        if position == 'BOT_DUO':
            df_merged = df_merged.drop_duplicates(subset=['matchId', 'minute', 'win'])
        else:
            df_merged = df_merged.drop_duplicates(subset=['matchId', 'minute', 'participantId'])

        print(f"   ✅ 병합 완료: {len(df_merged)}행")
        return df_merged

    except Exception as e:
        print(f"❌ 오류 발생 ({position}): {e}")
        return pd.DataFrame()

# ==========================================
# 2. 시각화 함수
# ==========================================
def plot_comprehensive_analysis(df, match_id, position, window_size=3):
    set_korean_font()

    global_mean = df.groupby('minute')['totalGold'].mean().to_dict()
    match_data = df[df['matchId'] == match_id].sort_values(['minute'])

    if match_data.empty:
        return

    win_data = match_data[match_data['win'] == True].sort_values('minute')
    lose_data = match_data[match_data['win'] == False].sort_values('minute')

    if win_data.empty or lose_data.empty:
        return

    common_minutes = sorted(list(set(win_data['minute']) & set(lose_data['minute'])))
    win_data = win_data[win_data['minute'].isin(common_minutes)]
    lose_data = lose_data[lose_data['minute'].isin(common_minutes)]
    minutes = win_data['minute'].values

    win_color = '#0059b3'
    lose_color = '#cc0000'
    bg_color = '#fafafa'

    def process_ratio(data, col_name):
        ratios = [val / global_mean[m] if m in global_mean and global_mean[m] > 0 else 1.0
                  for m, val in zip(data['minute'], data[col_name])]
        return pd.Series(ratios).rolling(window=window_size, min_periods=1).mean().values

    win_actual = process_ratio(win_data, 'totalGold')
    win_pred = process_ratio(win_data, 'predictedGold')
    lose_actual = process_ratio(lose_data, 'totalGold')
    lose_pred = process_ratio(lose_data, 'predictedGold')

    fig, ax = plt.subplots(figsize=(14, 8))
    ax.set_facecolor(bg_color)

    ax.axhline(y=1.0, color='black', linestyle='-', linewidth=2, alpha=0.5, label='서버 평균 (1.0)')
    ax.plot(minutes, lose_actual, color=lose_color, marker='o', linewidth=3, markersize=6, label='패배팀 실제')
    ax.plot(minutes, lose_pred, color=lose_color, linestyle='--', linewidth=2.5, alpha=0.7, label='패배팀 AI 예측')
    ax.plot(minutes, win_pred, color=win_color, linestyle='--', linewidth=2.5, alpha=0.7, label='승리팀 AI 예측')
    ax.plot(minutes, win_actual, color=win_color, linewidth=3, marker='o', markersize=6, label='승리팀 실제')

    ax.fill_between(minutes, win_actual, lose_actual, where=(win_actual >= lose_actual),
                    facecolor=win_color, alpha=0.15, interpolate=True, label='승리팀 우위')
    ax.fill_between(minutes, lose_actual, win_actual, where=(lose_actual > win_actual),
                    facecolor=lose_color, alpha=0.15, interpolate=True, label='패배팀 우위')
    ax.fill_between(minutes, win_actual, win_pred, where=(win_actual > win_pred),
                    facecolor='none', edgecolor=win_color, hatch='///', alpha=0.3)

    plt.title(f"[{position}]  기여도 분석 (Match: {match_id})", fontsize=20, fontweight='bold')
    plt.xlabel("게임 시간 (분)", fontsize=14)
    plt.ylabel("평균 대비 기여도 (Ratio)", fontsize=14)
    plt.legend(loc='upper left', fontsize=11, frameon=True, facecolor='white', framealpha=0.9, shadow=True)
    plt.grid(True, alpha=0.4, linestyle='--')

    y_vals = np.concatenate([win_actual, lose_actual])
    ax.set_ylim(y_vals.min() - 0.1, y_vals.max() + 0.1)

    plt.tight_layout()
    save_path = os.path.join(OUTPUT_DIR, f"{position}_analysis_{match_id}.png")
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()
    print(f"   📈 그래프 저장 완료: {save_path}")

# ==========================================
# 3. 메인 실행
# ==========================================
def main():
    positions = ['TOP', 'JUNGLE', 'MIDDLE', 'BOT_DUO']

    # 첫 번째 포지션에서 랜덤 matchId 선택
    df_first = load_and_merge_data(positions[0])
    if df_first.empty:
        print("❌ 데이터를 불러올 수 없습니다.")
        return

    target_match_id = 'KR_7467783009'
    print(f"\n🎯 선택된 경기: {target_match_id}")

    plot_comprehensive_analysis(df_first, target_match_id, positions[0], window_size=3)

    # 나머지 포지션
    for pos in positions[1:]:
        df = load_and_merge_data(pos)
        if not df.empty:
            plot_comprehensive_analysis(df, target_match_id, pos, window_size=3)

    print(f"\n✅ 모든 분석 완료. 결과는 '{OUTPUT_DIR}' 폴더를 확인하세요.")

if __name__ == "__main__":
    main()

📂 [TOP] Loading: test_predictions_TOP_20251207_171358.csv
   ✅ 병합 완료: 436396행

🎯 선택된 경기: KR_7467783009
   📈 그래프 저장 완료: output/visualize_performance/TOP_analysis_KR_7467783009.png
📂 [JUNGLE] Loading: test_predictions_JUNGLE_20251207_171358.csv
   ✅ 병합 완료: 436396행
   📈 그래프 저장 완료: output/visualize_performance/JUNGLE_analysis_KR_7467783009.png
📂 [MIDDLE] Loading: test_predictions_MIDDLE_20251207_171358.csv
   ✅ 병합 완료: 436396행
   📈 그래프 저장 완료: output/visualize_performance/MIDDLE_analysis_KR_7467783009.png
📂 [BOT_DUO] Loading: test_predictions_BOT_DUO_20251207_171358.csv
   ✅ 병합 완료: 436396행
   📈 그래프 저장 완료: output/visualize_performance/BOT_DUO_analysis_KR_7467783009.png

✅ 모든 분석 완료. 결과는 'output/visualize_performance' 폴더를 확인하세요.
